## Missing values and duplicates

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_train = pd.read_csv('../data/training.csv')
df_test = pd.read_csv('../data/test.csv')
df_id_table = pd.read_csv('../data/IdLookupTable.csv')

### missing values

In [4]:
df_train.isna().sum()

left_eye_center_x              10
left_eye_center_y              10
right_eye_center_x             13
right_eye_center_y             13
left_eye_inner_corner_x      4778
left_eye_inner_corner_y      4778
left_eye_outer_corner_x      4782
left_eye_outer_corner_y      4782
right_eye_inner_corner_x     4781
right_eye_inner_corner_y     4781
right_eye_outer_corner_x     4781
right_eye_outer_corner_y     4781
left_eyebrow_inner_end_x     4779
left_eyebrow_inner_end_y     4779
left_eyebrow_outer_end_x     4824
left_eyebrow_outer_end_y     4824
right_eyebrow_inner_end_x    4779
right_eyebrow_inner_end_y    4779
right_eyebrow_outer_end_x    4813
right_eyebrow_outer_end_y    4813
nose_tip_x                      0
nose_tip_y                      0
mouth_left_corner_x          4780
mouth_left_corner_y          4780
mouth_right_corner_x         4779
mouth_right_corner_y         4779
mouth_center_top_lip_x       4774
mouth_center_top_lip_y       4774
mouth_center_bottom_lip_x      33
mouth_center_b

In [5]:
df_test.isna().sum()

ImageId    0
Image      0
dtype: int64

There are a lot of missing values in the data. We can impute them using other models, or just ignore the value of loss function for them.

## duplicates

In [16]:
df_train[df_train.duplicated(subset="Image", keep=False)]

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
1906,63.749207,36.221939,28.384546,39.022015,56.949211,37.972917,70.765705,36.239131,35.453863,39.606916,...,57.014921,64.760120,72.939445,34.426676,74.638118,50.297134,73.254912,50.564068,73.934382,5 10 20 29 39 48 61 73 94 112 120 123 127 124 ...
1907,22.763345,55.617202,1.528527,56.404973,19.064954,56.291237,27.571879,56.384378,5.751046,56.743904,...,60.069687,22.923362,73.483945,2.245766,74.128316,12.605173,71.888816,12.536477,76.628863,5 10 20 29 39 48 61 73 94 112 120 123 127 124 ...
2294,68.147669,36.542446,32.530286,40.170137,NaN,NaN,NaN,NaN,NaN,NaN,...,67.213029,NaN,NaN,NaN,NaN,NaN,NaN,54.296469,79.085486,137 136 135 134 132 128 123 121 121 128 106 70...
2297,69.275782,35.639229,28.891200,38.888567,NaN,NaN,NaN,NaN,NaN,NaN,...,64.883287,NaN,NaN,NaN,NaN,NaN,NaN,51.172364,74.631302,56 57 57 56 54 51 50 53 55 56 55 54 55 65 75 8...
2298,63.366646,41.137231,34.119385,38.170092,NaN,NaN,NaN,NaN,NaN,NaN,...,58.939815,NaN,NaN,NaN,NaN,NaN,NaN,46.835692,70.808123,13 13 11 10 9 8 7 8 8 8 9 9 11 12 15 18 20 24 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7036,71.258174,39.744607,33.465219,36.687826,NaN,NaN,NaN,NaN,NaN,NaN,...,63.365209,NaN,NaN,NaN,NaN,NaN,NaN,47.915464,77.537562,247 244 241 238 233 227 218 200 185 171 157 14...
7041,65.711151,38.843545,32.268751,37.706043,NaN,NaN,NaN,NaN,NaN,NaN,...,64.095963,NaN,NaN,NaN,NaN,NaN,NaN,49.786191,78.200923,164 165 167 168 169 170 172 171 170 171 173 16...
7044,67.402546,31.842551,29.746749,38.632942,NaN,NaN,NaN,NaN,NaN,NaN,...,67.029093,NaN,NaN,NaN,NaN,NaN,NaN,50.426637,79.683921,71 74 85 105 116 128 139 150 170 187 201 209 2...
7045,66.134400,38.365501,30.478626,39.950198,NaN,NaN,NaN,NaN,NaN,NaN,...,66.626011,NaN,NaN,NaN,NaN,NaN,NaN,50.287397,77.983023,60 60 62 57 55 51 49 48 50 53 56 56 106 89 77 ...


In [19]:
df_train_duplicates = pd.concat([g for _, g in df_train.groupby("Image") if len(g) > 1])

In [20]:
df_train_duplicates

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
3648,67.552471,36.647602,29.803749,34.463794,NaN,NaN,NaN,NaN,NaN,NaN,...,65.349106,NaN,NaN,NaN,NaN,NaN,NaN,47.898185,75.332242,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 ...
6490,67.947223,36.352608,30.106793,34.453179,NaN,NaN,NaN,NaN,NaN,NaN,...,65.983666,NaN,NaN,NaN,NaN,NaN,NaN,48.952951,81.938880,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 ...
5741,65.629689,41.037031,35.407413,38.370364,NaN,NaN,NaN,NaN,NaN,NaN,...,61.185191,NaN,NaN,NaN,NaN,NaN,NaN,49.925920,74.814809,0 2 6 9 11 10 9 10 11 14 12 8 6 6 12 25 54 80 ...
6951,67.305956,39.818062,32.790116,38.570507,NaN,NaN,NaN,NaN,NaN,NaN,...,61.026649,NaN,NaN,NaN,NaN,NaN,NaN,49.840160,75.997404,0 2 6 9 11 10 9 10 11 14 12 8 6 6 12 25 54 80 ...
6306,69.133029,34.378435,30.913770,34.378435,NaN,NaN,NaN,NaN,NaN,NaN,...,64.465945,NaN,NaN,NaN,NaN,NaN,NaN,50.226689,78.289945,0 36 43 27 5 12 31 38 29 12 55 118 122 103 92 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4684,66.741055,37.884447,29.824691,40.837756,NaN,NaN,NaN,NaN,NaN,NaN,...,67.786691,NaN,NaN,NaN,NaN,NaN,NaN,49.021200,81.445745,98 83 78 81 89 101 116 133 149 162 169 168 170...
5537,67.247165,38.846421,31.437078,38.526699,NaN,NaN,NaN,NaN,NaN,NaN,...,69.221009,NaN,NaN,NaN,NaN,NaN,NaN,45.185530,80.731367,98 98 98 98 99 99 101 98 73 51 37 23 23 24 26 ...
6109,68.701774,40.692772,33.616696,39.260723,NaN,NaN,NaN,NaN,NaN,NaN,...,68.856230,NaN,NaN,NaN,NaN,NaN,NaN,45.550330,80.789906,98 98 98 98 99 99 101 98 73 51 37 23 23 24 26 ...
2440,70.973696,34.803904,38.079168,36.201152,NaN,NaN,NaN,NaN,NaN,NaN,...,66.009600,NaN,NaN,NaN,NaN,NaN,NaN,56.243712,73.927488,99 96 103 103 115 125 126 124 130 142 131 117 ...
